In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'smartphones:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5018553%2F8427893%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240516%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240516T065625Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D38544fd5ecb5501ecc02752307f09dc5c695a5e6af868f4fb5a33c8843846fed6a6ec9e031714d2ea10131c0cf0d8f5611cd7c88e9bf0cd3f528ffdb05c7882b50211c984b1e14192f99392e8aaa392b71fb28590c3e59aa877a29b5e2361295f351e80903911cd764f296b21516d3fadc12ec01d1945a1ab1795deed38894cd54447cbf109408a15f4b12236fc0d8fc6281635add52f8b5c4b69c1a9f9e5d51f8a916a681347c8e46db9fe1cb6f16b13979a71d1a710dc4725b5361976a50efd72d36b80bff3d9550b423d420a5f258451292fa64cdc7442d18f5729a8fc32618c55ab645797cae4e85c914fe86e14e13bf98b09ad6186d85542f70b42d63b6'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 28006 bytes downloaded
Downloaded and uncompressed: smartphones
Data source import complete.


In [7]:
# Импортируем необходимые библиотеки
!pip install gmdh
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingRegressor
from sklearn.neural_network import MLPRegressor
from gmdh import Combi, Multi, Mia, Ria




df = pd.read_csv('/kaggle/input/smartphones/smartphones.csv')

# Просмотр первых строк данных для понимания их структуры
print(df.head())

# Предварительная обработка данных

# Кодирование категориальных признаков
label_encoders = {}
categorical_columns = ['Brand', 'Model', 'Color', 'Free']

for col in categorical_columns:
    encoder = LabelEncoder()
    df[col] = encoder.fit_transform(df[col])
    label_encoders[col] = encoder

# Заполнение/удаление пропусков (если есть)
df = df.dropna()

# Отделяем признаки от целевой переменной
X = df[['Brand', 'Model', 'RAM', 'Storage', 'Color', 'Free']]
y = df['Final Price']

# Стандартизация (нормализация) признаков
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Разделение данных на обучающую и тестовую выборки
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение моделей

# 1. Модель стекинга
estimators = [('lr', LinearRegression()), ('mlp', MLPRegressor(max_iter=2000))]
stacking_regressor = StackingRegressor(estimators=estimators, final_estimator=LinearRegression())
stacking_regressor.fit(x_train, y_train)
stacking_predictions = stacking_regressor.predict(x_test)

# 2. Модель многослойного персептрона (MLP)
mlp_regressor = MLPRegressor(max_iter=2000, learning_rate_init=0.001, tol=1e-4)
mlp_regressor.fit(x_train, y_train)
mlp_predictions = mlp_regressor.predict(x_test)

# 3. Метод COMBI (линейная модель)
combi_model = Combi()
combi_model.fit(x_train, y_train)
combi_predictions = combi_model.predict(x_test)

# 4. Метод MIA (нелинейная модель)
mia_model = Mia()
mia_model.fit(x_train, y_train)
mia_predictions = mia_model.predict(x_test)

# Оценка качества моделей
metrics = {
    'StackingRegressor': mean_absolute_error(y_test, stacking_predictions),
    'MLPRegressor': mean_absolute_error(y_test, mlp_predictions),
    'COMBI': mean_absolute_error(y_test, combi_predictions),
    'MIA': mean_absolute_error(y_test, mia_predictions)
}

# Вывод метрик
for model_name, mae in metrics.items():
    print(f'{model_name} MAE: {mae}')

# В дополнение, можно также вывести RMSE (среднеквадратичную ошибку)
for model_name, preds in zip(metrics.keys(), [stacking_predictions, mlp_predictions, combi_predictions, mia_predictions]):
    rmse = mean_squared_error(y_test, preds, squared=False)
    print(f'{model_name} RMSE: {rmse}')


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()


                                     Smartphone     Brand           Model  \
0            Realme C55 8/256GB Sunshower Libre    Realme             C55   
1      Samsung Galaxy M23 5G 4/128GB Azul Libre   Samsung      Galaxy M23   
2  Motorola Moto G13 4/128GB Azul Lavanda Libre  Motorola        Moto G13   
3      Xiaomi Redmi Note 11S 6/128GB Gris Libre    Xiaomi  Redmi Note 11S   
4       Nothing Phone (2) 12/512GB Blanco Libre   Nothing       Phone (2)   

    RAM  Storage   Color Free  Final Price  
0   8.0    256.0  Yellow  Yes       231.60  
1   4.0    128.0    Blue  Yes       279.00  
2   4.0    128.0    Blue  Yes       179.01  
3   6.0    128.0    Gray  Yes       279.99  
4  12.0    512.0   White  Yes       799.00  


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perce

StackingRegressor MAE: 116.84958678655637
MLPRegressor MAE: 115.67861153835231
COMBI MAE: 115.82834729028309
MIA MAE: 4.433690975720055e+160
StackingRegressor RMSE: 171.277668690201
MLPRegressor RMSE: 168.31891826496238
COMBI RMSE: 166.80217892973366
MIA RMSE: inf


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:446: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0, weights=sample_weight)
